In [4]:
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn. functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import numpy as np
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.optim.lr_scheduler import LinearLR

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
CONFIGURATION= {'LEARNING_RATE': 0.001,
                'EPOCHS': 50,
                'ADAM_BETAS': (0.9, 0.999),
                'ADAM_EPS': 1e-07,
                'SEED': 42,
                'BATCH_SIZE': 16,
                'LINEAR_LR': False,
                'DROPOUTS': 0.3,
                'L2_REGULARISATION': 0.001
                }

In [11]:
from clearml import Task, Logger

task= Task.init(project_name='profanity_detection', task_name='adima_training_classifier3')

ClearML Task: overwriting (reusing) task id=fc717fc84b9847dda38082a9dad51113
ClearML results page: http://13.233.63.202:8080/projects/1d29852235204d9098563d5acdf7d416/experiments/fc717fc84b9847dda38082a9dad51113/output/log


In [12]:
task.connect(CONFIGURATION)

{'LEARNING_RATE': 0.001,
 'EPOCHS': 50,
 'ADAM_BETAS': (0.9, 0.999),
 'ADAM_EPS': 1e-07,
 'SEED': 42,
 'BATCH_SIZE': 16,
 'LINEAR_LR': False,
 'DROPOUTS': 0.3,
 'L2_REGULARISATION': 0.001}

In [23]:
task.close()

### Data Pipeline

In [13]:
def time_shift(aud, shift_limit):
    sig, sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)


def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
        aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
        aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
    return aug_spec

class AudioDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd. read_csv (csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self) :
        return len(self.annotations)

    def __getitem__(self, index) :

        file_path = os.path.join(self. root_dir, self.annotations. iloc[index, 0])
        #print(file_path)
        audio_feature= np.load(file_path)
        
        y_label = self .annotations. iloc [index, 1]
        
        if self.transform:
            audio_feature = self.transform(audio_feature)
        
        audio_feature=torch.from_numpy(audio_feature)

        audio_feature, sr = time_shift((audio_feature, 16000), 0.1)
        audio_feature = audio_feature.unsqueeze(0)
        audio_feature = spectro_augment(audio_feature, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
        audio_feature = audio_feature.squeeze()

        max_pool, _ = torch.max(audio_feature, dim=1)

        return (max_pool, y_label)
    

combined_dataset = AudioDataset (csv_file = str(Path.home())+'/Daniyal/profanity_detection/data/features/train_val_oversampled.csv', root_dir =str(Path.home())+ "/Daniyal/profanity_detection/data/features/")
combined_loader = DataLoader (dataset=combined_dataset, batch_size=CONFIGURATION['BATCH_SIZE'], shuffle=True)#, sampler=train_sampler)


test_dataset = AudioDataset (csv_file = str(Path.home())+'/Daniyal/profanity_detection/data/features/test/divided_test.csv', root_dir =str(Path.home())+ "/Daniyal/profanity_detection/data/features/test/")
test_loader = DataLoader(dataset=test_dataset, batch_size=CONFIGURATION['BATCH_SIZE'], shuffle=True)


val_dataset = AudioDataset (csv_file = str(Path.home())+'/Daniyal/profanity_detection/data/features/test/divided_test.csv', root_dir =str(Path.home())+ "/Daniyal/profanity_detection/data/features/test/")
val_loader = DataLoader (dataset=val_dataset, batch_size=CONFIGURATION['BATCH_SIZE'], shuffle=True)

### Model

In [18]:
#Sequential model
class NN(nn.Module): #inherit n module
    def __init__(self) :
        super(NN, self).__init__()
        self.fc1 = nn. Linear (512, 256)
        self.d1= nn. Dropout (p=CONFIGURATION['DROPOUTS'])
        self. fc2 = nn. Linear (256, 128)
        self.d2= nn. Dropout (p=0.1)
        self. fc3 = nn. Linear (128, 2)

        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')


    def forward (self, x):
        x = F.gelu(self.fc1(x) ) 
        x = self.d2(F.gelu(self. fc2(x) ) )
        x= self. fc3(x)
        return x

In [19]:
model= NN().to(device)
print(model)

NN(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (d1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (d2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


### Training

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),
                        lr=CONFIGURATION['LEARNING_RATE'], 
                        betas=CONFIGURATION['ADAM_BETAS'], 
                        eps=CONFIGURATION['ADAM_EPS'], 
                        weight_decay=CONFIGURATION['L2_REGULARISATION'])
scheduler = LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=50)

train_accuracy_list=[]
val_accuracy_list=[]
train_precision_list=[]
val_precision_list=[]
train_recall_list=[]
val_recall_list=[]
loss_list=[]
val_loss_list=[]

In [21]:
for epoch in range(CONFIGURATION['EPOCHS']):
  model.train()

  print(f"epoch: {epoch+1}/{CONFIGURATION['EPOCHS']}")
  
  pred=torch.tensor([]).to(device=device)
  label=torch.tensor([]).to(device=device)
  running_loss = 0.0

  for batch_idx, (data, targets) in enumerate(combined_loader):

    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape (data.shape [0], -1)

    scores = model(data)
    loss = criterion(scores, targets) 

    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    running_loss += loss.item()
    _, prediction = torch.max(scores, 1)
    pred=torch.cat([pred, prediction])
    label=torch.cat([label, targets])

  num_batches = len(combined_loader)
  avg_loss = running_loss / num_batches

  loss_list.append(avg_loss)


  train_acc=accuracy_score(label.cpu().detach().numpy(), pred.cpu().detach().numpy())
  train_accuracy_list.append(train_acc)
  train_precision= precision_score(label.cpu().detach().numpy(), pred.cpu().detach().numpy(), zero_division=0, pos_label=1)
  train_precision_list.append(train_precision)
  train_recall=recall_score(label.cpu().detach().numpy(), pred.cpu().detach().numpy(), pos_label=1)
  train_recall_list.append(train_recall)
  macro_f1=f1_score(label.cpu().detach().numpy(), pred.cpu().detach().numpy(), average='macro')

  #print(f'Training: Epoch: {epoch}, loss: {avg_loss:.2f}, train_accuracy: {accuracy(pred, label):.2f}, train_precision: {precision(pred, label): .2f}, train_recall: {recall(pred, label): .2f},')
  print(f'Training: Epoch: {epoch}, loss: {avg_loss}, train_accuracy: {train_acc:.2f}, train_precision: {train_precision: .2f}, train_recall: {train_recall: .2f}, train_f1_score: {macro_f1}, ')


  val_pred=torch.tensor([]).to(device=device)
  val_label=torch.tensor([]).to(device=device)
  val_running_loss = 0.0

  model.eval()
  with torch.no_grad(): 
    for batch_idx, (data, targets) in enumerate(val_loader):

      data = data.to(device=device)
      targets = targets.to(device=device)

      #data = data.reshape (data.shape [0], -1)

      scores = model(data)
      val_loss = criterion(scores, targets)
      val_running_loss += val_loss.item()

      _, prediction = torch.max(scores, 1)

      val_pred=torch.cat([val_pred, prediction])
      val_label=torch.cat([val_label, targets])


    num_val_batches = len(val_loader)
    avg_val_loss = val_running_loss / num_val_batches
    val_loss_list.append(avg_val_loss)

    val_acc=accuracy_score(val_label.cpu().detach().numpy(), val_pred.cpu().detach().numpy() )
    val_accuracy_list.append(val_acc)
    val_precision=precision_score(val_label.cpu().detach().numpy(), val_pred.cpu().detach().numpy(), zero_division=0, pos_label=1 )
    val_precision_list.append(val_precision)
    val_recall=recall_score(val_label.cpu().detach().numpy(), val_pred.cpu().detach().numpy(), pos_label=1 )
    val_recall_list.append(val_recall)
    macro_f1 = f1_score(val_label.cpu().detach().numpy(), val_pred.cpu().detach().numpy(), average='macro')

    #print(f'val_accuracy: {accuracy(val_pred, val_label):.2f}, val_precision: {precision(val_pred, val_label): .2f}, val_recall: {recall(val_pred, val_label): .2f},')
    print(f"val_loss: {avg_val_loss}, val_accuracy: {val_acc:.2f}, val_precision: {val_precision: .2f}, val_recall: {val_recall: .2f}, val_f1_score: {macro_f1}, lr: {optimizer.param_groups[0]['lr']}")

  Logger.current_logger().report_scalar("Loss", "train", iteration=epoch, value=avg_loss)
  Logger.current_logger().report_scalar("Loss", "val", iteration=epoch, value=avg_val_loss)
  Logger.current_logger().report_scalar("Accuracy", "train", iteration=epoch, value=train_acc)
  Logger.current_logger().report_scalar("Accuracy", "val", iteration=epoch, value=val_acc)
  Logger.current_logger().report_scalar("Precision", "train", iteration=epoch, value=train_precision)
  Logger.current_logger().report_scalar("Precision", "val", iteration=epoch, value=val_precision)
  Logger.current_logger().report_scalar("Recall", "train", iteration=epoch, value=train_recall)
  Logger.current_logger().report_scalar("Recall", "val", iteration=epoch, value=val_precision)

  if CONFIGURATION['LINEAR_LR']:
    scheduler.step()
  


epoch: 1/50
Training: Epoch: 0, loss: 0.5928935404144117, train_accuracy: 0.67, train_precision:  0.65, train_recall:  0.73, train_f1_score: 0.66875, 
val_loss: 0.5346122036377589, val_accuracy: 0.69, val_precision:  0.53, val_recall:  0.90, val_f1_score: 0.688663282571912, lr: 0.001
epoch: 2/50
Training: Epoch: 1, loss: 0.5326707478779465, train_accuracy: 0.73, train_precision:  0.71, train_recall:  0.78, train_f1_score: 0.726622492916583, 
val_loss: 0.4803905685742696, val_accuracy: 0.73, val_precision:  0.57, val_recall:  0.84, val_f1_score: 0.721819061441703, lr: 0.001
epoch: 3/50
Training: Epoch: 2, loss: 0.5116511365370964, train_accuracy: 0.75, train_precision:  0.74, train_recall:  0.77, train_f1_score: 0.7498822425296352, 
val_loss: 0.5302209556102753, val_accuracy: 0.70, val_precision:  0.54, val_recall:  0.87, val_f1_score: 0.6978653530377668, lr: 0.001
epoch: 4/50
Training: Epoch: 3, loss: 0.4893513886786219, train_accuracy: 0.74, train_precision:  0.72, train_recall:  0.78

In [60]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d%m%Y_%H%M")

torch.save(model.state_dict(),f'/home/ubuntu/Daniyal/work/model/model_trained_on_{dt_string}.pt')

In [22]:
#test performance
model.eval()
test_pred=torch.tensor([]).to(device=device)
test_label=torch.tensor([]).to(device=device)

model.eval()
with torch.no_grad(): 
    for batch_idx, (data, targets) in enumerate(test_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        scores = model(data)

        _, prediction = torch.max(scores, 1)

        test_pred=torch.cat([test_pred, prediction])
        test_label=torch.cat([test_label, targets])

    test_acc=accuracy_score(test_label.cpu().detach().numpy(), test_pred.cpu().detach().numpy() )
    test_precision=precision_score(test_label.cpu().detach().numpy(), test_pred.cpu().detach().numpy(), zero_division=0 )
    test_recall=recall_score(test_label.cpu().detach().numpy(), test_pred.cpu().detach().numpy() )

    #print(f'val_accuracy: {accuracy(val_pred, val_label):.2f}, val_precision: {precision(val_pred, val_label): .2f}, val_recall: {recall(val_pred, val_label): .2f},')
    print(f"test_accuracy: {val_acc:.2f}, test_precision: {val_precision: .2f}, test_recall: {val_recall: .2f}")


test_accuracy: 0.73, test_precision:  0.65, test_recall:  0.49
